In [1]:
import pandas as pd

### Read-in data

In [2]:
path = '../Data/RAWDATA_Ward_Scenarios.xlsx'
df_wards = pd.read_excel(path)

In [3]:
df_wards.head()

,block,id,type,max_no_beds,A_color,A_no_beds,B_color,B_no_beds,C_color,C_no_beds,normal_use
0,ATC,ADD L2,Day surgery,35,excl.,0,excl.,0,excl.,0,NaN
1,ATC,ADD L4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients
2,ATC,ADD M4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients
3,ATC,ADD L5,GIM,32,G,32,G,32,R,22,Elective surgery patients
4,ATC,ADD M5,GIM,32,G,32,G,32,R,22,Elective surgery patients


### Mark wards that change between scenarios

In [4]:
df_wards['AB_change'] = df_wards.A_color != df_wards.B_color
df_wards['BC_change'] = df_wards.B_color != df_wards.C_color
# df_wards.head()

### Assign 'taxi-rank' numbers to changing wards

In [5]:
import numpy as np

def _shuffled_range(n):
    """
    Generates a randomly shuffled range from 0 to n-1
    """
    shuffled_range = np.arange(n)
    np.random.shuffle(shuffled_range)
    return shuffled_range

In [6]:
df_wards['AB_change_no'] = None

change_no = _shuffled_range(len(df_wards[df_wards.AB_change]))
df_wards.loc[df_wards.AB_change, 'AB_change_no'] = change_no

# df_wards[df_wards.AB_change]

In [7]:
df_wards['BC_change_no'] = None

change_no = _shuffled_range(len(df_wards[df_wards.BC_change]))
df_wards.loc[df_wards.BC_change, 'BC_change_no'] = change_no

# df_wards[df_wards.BC_change]

### View updated dataframe

In [8]:
df_wards.head()

,block,id,type,max_no_beds,A_color,A_no_beds,B_color,B_no_beds,C_color,C_no_beds,normal_use,AB_change,BC_change,AB_change_no,BC_change_no
0,ATC,ADD L2,Day surgery,35,excl.,0,excl.,0,excl.,0,NaN,False,False,None,None
1,ATC,ADD L4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients,False,False,None,None
2,ATC,ADD M4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients,False,False,None,None
3,ATC,ADD L5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True,None,10
4,ATC,ADD M5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True,None,7


### Summarise number of beds

In [9]:
df_wards.groupby(by='A_color').agg(No_Wards=('A_no_beds', 'count'), No_Beds=('A_no_beds', 'sum'))

,No_Wards,No_Beds
A_color,,
A,6,95
DECANT,1,0
G,23,550
R,4,84
RG,1,23
excl.,12,0


## __draft: dataframe wrangling

### Melt dataframe to have row for ward in each scenario

In [10]:
df_melt = df_wards.melt(id_vars=['block', 'id', 'max_no_beds', 'A_no_beds', 'B_no_beds', 'C_no_beds', 'AB_change', 'BC_change'], 
              value_vars=['A_color', 'B_color', 'C_color'], 
              var_name='scenario',
              value_name='color'
             )

### Change scenario column to only contain letter

In [11]:
df_melt.loc[:,'scenario'] = df_melt.scenario.apply(lambda s: s[0])

### Change three no_beds columns into one column (scenario comes from scenario column now)

#### Create new column

In [12]:
df_melt['no_beds'] = None
for scenario in df_melt.scenario.unique():
    col = scenario + '_no_beds'
    df_melt.loc[df_melt.scenario == scenario,'no_beds'] = df_melt.loc[df_melt.scenario == scenario,col]

#### Drop old columns

In [13]:
df_melt = df_melt.drop(columns=['A_no_beds', 'B_no_beds', 'C_no_beds'])

### View melted dataframe

In [14]:
df_melt.head()

,block,id,max_no_beds,AB_change,BC_change,scenario,color,no_beds
0,ATC,ADD L2,35,False,False,A,excl.,0
1,ATC,ADD L4,32,False,False,A,G,32
2,ATC,ADD M4,32,False,False,A,G,32
3,ATC,ADD L5,32,False,True,A,G,32
4,ATC,ADD M5,32,False,True,A,G,32


### View all wards that change

In [15]:
df_melt[df_melt.AB_change | df_melt.BC_change].sort_values(by=['id', 'scenario']).head()

,block,id,max_no_beds,AB_change,BC_change,scenario,color,no_beds
35,C&D,ADD C10,16,False,True,A,G,13
82,C&D,ADD C10,16,False,True,B,G,13
129,C&D,ADD C10,16,False,True,C,R,9
23,C&D,ADD C4,29,False,True,A,G,29
70,C&D,ADD C4,29,False,True,B,G,29


### View all wards for each scenario

In [16]:
df_melt.pivot(index='id', columns='scenario', values=['color', 'no_beds']).head()

color               no_beds        
scenario           A      B      C       A   B   C
id                                                
ADD A2 (NCCU)  excl.  excl.  excl.       0   0   0
ADD A3             G      G      G       8   8   8
ADD A4             G      G      G      26  26  26
ADD A5             G      G      G      28  28  28
ADD C10            G      G      R      13  13   9